In [55]:
import torch
import numpy as np
from konlpy.tag import Okt
import re
import pickle
from torch import nn

In [56]:
# 문장 분류하는 모델
class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, n_classes ,dropout=0.5, bidirectional=True, model_type="lstm"):
        super().__init__()  # 부모클래스 상속

        self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim, padding_idx=0)

        # rnn모델 일 경우
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )
        # lstm모델 일 경우
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )

        # bidirectional은 양방향성을 의미하는 파라미터
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, n_classes)   # 양방향일때 타임스탭에서 양방향의 정보(순방향,역방향)의 출력들을 결합하여 분류기에 전달
        else:
            self.classifier = nn.Linear(hidden_dim, n_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings) 
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [57]:
# 1. 새로운 리뷰 데이터 불러오기
new_reviews = ["1950년대 한국전쟁 직후 가난했지만 낭만이 있던 시대! 초괴의 국극 배우에 도전하는 '타고난 소리천재' 정년이를 둘러싼 경쟁과 연대, 그리고 찬란한 성장기"]

In [58]:
# 2. 한글만 남기고 정규식 적용
def re_text(text):
    text = re.sub(r'[^\n가-힇\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [59]:
new_reviews = [re_text(review) for review in new_reviews]

In [60]:
# 3. Okt 토크나이저 로드 및 토큰화
tokenizer = Okt()
STOP_PATH = 'kor_stopwords.txt'

def make_stopwords(STOP_PATH):
    with open(STOP_PATH, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return set(stopwords)

In [61]:
stopwords = make_stopwords(STOP_PATH)
new_tokens = [[token for token in tokenizer.morphs(review) if token not in stopwords] for review in new_reviews]

In [62]:
# 4. 단어 사전 로드
with open('vocab.pkl', 'rb') as f:
    vocab_list = pickle.load(f)

# 리스트를 사전으로 변환
token_to_id = {token: idx for idx, token in enumerate(vocab_list)}

unk_id = token_to_id.get("<unk>", 1)
pad_id = token_to_id.get("<pad>", 0)

In [63]:
# 5. 정수 인코딩
new_ids = [[token_to_id.get(token, unk_id) for token in tokens] for tokens in new_tokens]

In [64]:
# 6. 패딩 적용
max_length = 60
def pad_sequences(sequences, max_length, pad_value):
    result = []
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [65]:
new_ids_padded = pad_sequences(new_ids, max_length, pad_id)

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('best_model.pth', map_location=device)
model = model.to(device)
model.eval()

C:\Users\KDP15\AppData\Local\Temp\ipykernel_4516\2756406480.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_model.pth', map_location=device)


SentenceClassifier(
  (embedding): Embedding(5002, 128, padding_idx=0)
  (model): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (classifier): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [67]:
# 8. 모델 예측
new_ids_tensor = torch.tensor(new_ids_padded).to(device)  # 데이터를 모델이 있는 디바이스로 이동

with torch.no_grad():
    outputs = model(new_ids_tensor)
    predictions = torch.sigmoid(outputs)

# 9. 예측 결과 출력 (0.5 이상이면 긍정, 미만이면 부정)
for i, review in enumerate(new_reviews):
    prediction = 1 if predictions[i] >= 0.5 else 0
    print(f"줄거리: {review}")
    print(f"예측된 등급: {'15세' if prediction == 1 else '나머지'}")

줄거리: 년대 한국전쟁 직후 가난했지만 낭만이 있던 시대 초괴의 국극 배우에 도전하는 타고난 소리천재 정년이를 둘러싼 경쟁과 연대 그리고 찬란한 성장기
예측된 등급: 15세
